This notebook is a narrative exploration of efforts to predict the number of citations per year a paper will receive,
based on data available at time of publication.

A full exploration of 25 different prediction models can be found [here](askfhs)

In this notebook I look to predict the number of citations a paper will received based upon
* the words used in the abstract
* physics-inspired semantic metrics (implemented in github.com/zhafen/cc and employed in Imel & Hafen in prep)
* metadata (publishing journal, number of authors, number of pages, etc.)

The raw code for this analysis can be [found here](https://github.com/zhafen/work-sample).

I aimed to keep this work sample clean, so please reach out if you have questions about details I have not included.

You can jump to punchlines in the analysis via these links:

# User-Defined Parameters

This dictionary contains various user-defined parameters. They will be explained when they are utilized.

In [24]:
pm = dict(
    # Data
    data_dir = '/Users/zhafen/data/literature_topography',
    region_number = 8,
    convergence_degree = 3,
    kernel_size = 16,
    # Features
    numerical_variables = [
        'age',
        'converged_kernel_size',
        'density',
        'fringe_factor',
        'references_count',
        'page_count',
        'log_author_count',
    ],
    categorical_variables = [
        'journal_filtered',
    ],
    semantic_variables = [
        'density',
        'fringe_factor',
    ]
)

# Data

I use publication abstracts and metadata pulled from the [NASA astrophysics data sytem](https://ui.adsabs.harvard.edu) via [the official API](https://ui.adsabs.harvard.edu/help/api/). The analyzed publications are from a randomly-chosen physics or astrophysics specialization.

I externally preprocessed the abstract data with natural language processing (including tokenizing, stemming, and removing filler words), and each abstract has a corresponding bag-of-words representation.

## Load Raw Data

In [26]:
# Imports
import numpy as np
import os

# My custom non-relational-data-management package
import verdict
# My library for NLP analysis of scientific abstracts
from cc import atlas, cartography, utils

In [16]:
# Load summary information.
# I analyzed several randomly chosen specializations ("regions"), of which we are choosing an arbitrary one.
summary_data_fp = os.path.join( pm['data_dir'], 'regions', 'regions_summary.h5' )
data = verdict.Dict.from_hdf5( summary_data_fp )
data_k = data['regions'][str(pm['region_number'])]

In [18]:
# Class for management of abstracts
atlas_dir = os.path.join( pm['data_dir'], 'regions', 'region_{}'.format( pm['region_number'] ) )
a = atlas.Atlas( atlas_dir, load_bibtex=False )

Loading saved atlas data.


0it [00:00, ?it/s]
100%|█████████████████████████████████| 37311/37311 [00:01<00:00, 21616.01it/s]


In [19]:
# Class for analysis of vectorized abstracts
projection = a.vectorize(
    verbose = True,
)
c = cartography.Cartographer( **projection )

Vectorizing text...
Using saved vectorized text...


In [23]:
# Retrieve metrics I calculated in external pre-processing
metrics_fp = os.path.join( atlas_dir, 'topography_metrics.h5' )
metrics = verdict.Dict.from_hdf5( metrics_fp )

In [27]:
# Not all the publications are viable for analysis.
# I've saved information about what publications are viable, and here we load the identifying information.
converged_kernel_size = data_k['converged_kernel_size'][:,-pm['convergence_degree']]
converged = converged_kernel_size >= pm['kernel_size']
publications = c.publications[converged]
inds = np.arange( c.publications.size )[converged]

# Cross Validation

In [282]:
from sklearn.model_selection import cross_validate

In [ ]:
cross_validate(
    estimator = model,
    X = x_t_df.values,
    y = df_train['log_citations_per_year'].values,
    cv = kfold,
    scoring = 'neg_mean_squared_error',
    return_estimator = True,
)

{'fit_time': array([0.0011487 , 0.00084901, 0.00049782, 0.000458  , 0.00043535]),
 'score_time': array([0.00037026, 0.00032496, 0.00028419, 0.00027323, 0.00028801]),
 'estimator': [Baseline(), Baseline(), Baseline(), Baseline(), Baseline()],
 'test_score': array([-0.42891344, -0.42412764, -0.38014943, -0.41732149, -0.44537275])}

# Credits

Utilized python packages include:
* [ads](https://github.com/andycasey/ads)
* nltk